# 함수 다루기

### 람다 추상화


특정 표현식으로부터 익명함수를 선언하는 것을 **람다 추상화**라 부른다. 

표현식 `t`가 유형 T2를 갖는다고 가정하자.
그러면 다음 표현식은 어떤 유형의 값 `x`과 함께 적용하면 `t`를 계산하는 함수를 표현한다.

```haskell
\x -> t
```

위 함수는`T1 -> T2` 유형을 갖는다. 

여기서 `T1`은 하스켈 컴파일러에 의해 추정되어 선언되며, 가장 일반화된 유형이 사용된다.

**주의:** 람다(lambda)는 그리스 알파벳 기호인 $\lambda$를 가리킨다. 
람다 추상화라고 부르는 이유는 위 함수를 책에서는 보통 다음과 같이 $\lambda$ 기호를 이용해서
표현하기 때문이다. 

$$\lambda x. t$$

#### 예제

아래 함수의 유형을 확인해 보자.

In [1]:
:t (\x -> [x, 'b'])

(\x -> [x, 'b']) :: Char -> [Char]

리스트에 포함된 항목들의 유형이 동일해야 하므로 `x`의 유형은 `Char`로 선언된다.
익명함수 사용법은 일반적 방식과 동일하다. 

In [2]:
(\x -> [x, 'b']) 'a'

"ab"

#### 예제

아래 함수의 유형을 확인해 보자.

In [3]:
:t (\x -> x^2 + 1)

(\x -> x^2 + 1) :: forall a. Num a => a -> a

위 정의에서 사용된 `x`는 사칙연산에 사용되는 값이어야 한다.
사칙연산을 허용하는 유형은 모두 `Num` 클래스의 인스턴스이다. 
따라서 `x`의 유형은 모든 `Num` 클래스의 인스턴스가 될 수 있다.

In [4]:
(\x -> x^2 + 1) 3

10

In [5]:
(\x -> x^2 + 1) 3.5

13.25

#### 예제

람다 추상화를 연속적으로 적용할 수도 있다. 
예를 들어, 두 수의 합의 제곱을 계산하는 함수를 익명으로 정의하면 다음과 같다. 

In [6]:
(\x -> (\y -> (x + y)^2)) 2 3

Line 1: Collapse lambdas
Found:
\ x -> (\ y -> (x + y) ^ 2)
Why not:
\ x y -> (x + y) ^ 2Line 1: Redundant bracket
Found:
\ x -> (\ y -> (x + y) ^ 2)
Why not:
\ x -> \ y -> (x + y) ^ 2

25

위 경고문은 연속적인 람다 추상화를 보다 간단하게 적용할 수 있음을 보여 준다.

람다 추상화를 인자에 적용하는 방식 또한 좌결합성을 사용한다.

In [7]:
(\x -> \y -> (x + y)^2) 2 3

Line 1: Collapse lambdas
Found:
\ x -> \ y -> (x + y) ^ 2
Why not:
\ x y -> (x + y) ^ 2

25

하지만 다음 방식을 하스켈에서 추천한다.

In [8]:
(\x y -> (x + y)^2) 2 3

25

### 함수의 부분적용

`add` 함수의 유형을 괄호를 사용하여 다시 정의하면 다음과 같다. 

In [9]:
add :: Integer -> (Integer -> Integer)
add x y = x + y

`add` 함수는 정수 한 개와 함께 적용되면 `Integer -> Integer` 유형의 값,
즉, 하나의 함수를 생성해야 한다.

예를 들어, `add 2`에 의해 생성된 함수를 `add2`라 한 후에 유형을 확인하면 다음과 같다. 

In [10]:
add2 = add 2
:t add2

add2 :: Integer -> Integer

그리고 `add2`를 정수 `n`과 함께 적용하면 `2 + n`을 계산한다. 

In [11]:
add2 3

5

In [12]:
add 2 3

5

이렇게 여러 개의 인자를 사용하는 함수들을 특정 인자 몇 개에만
부분적으로 적용하는 것을 부분적용이라 부르며 아래 결과가 성립한다. 

```haskell
add2 3 = (add 2) 3 = add 2 3
```

#### 예제: `addExp` 함수의 부분적용

`addExp` 함수의 유형을 괄호를 사용하여 다시 정의하면 다음과 같다. 

In [13]:
addExp :: Integer -> (Integer -> (Integer -> Integer))
addExp x y z = (x + y)^z

`addExp` 함수는 정수 한 개와 함께 적용되면 `Integer -> Integer -> Integer` 유형의 값,
즉, 하나의 함수를 생성해야 한다.

예를 들어, `addExp 2`에 의해 생성된 함수를 `addExp2`라 한 후에 유형을 확인하면 다음과 같다. 

In [14]:
addExp2 = addExp 2
:t addExp2

addExp2 :: Integer -> Integer -> Integer

이제 `addExp2` 함수를 정수 한 개와 함께 적용되면 `Integer -> Integer` 유형의 값,
즉, 또다른 하나의 함수를 생성해야 한다.

예를 들어, `addExp2 3`에 의해 생성된 함수를 `addExp23`라 한 후에 유형을 확인하면 다음과 같다. 

In [15]:
addExp23 = addExp2 3
:t addExp23

addExp23 :: Integer -> Integer

그리고 `addExp23`를 정수 `n`과 함께 적용하면 `(2 + 3)^n`, 즉, `5^n`을 계산한다. 

In [16]:
addExp23 3

125

물론 처음부터 다음과 같이 해도 동일한 결과를 얻는다. 

In [17]:
addExp2 3 3

125

In [18]:
addExp 2 3 3

125

이렇게 여러 개의 인자를 사용하는 함수들을 특정 인자 몇 개에만
부분적으로 적용하는 것을 부분적용이라 부르며 아래 결과가 성립한다. 

```haskell
addExp23 5 = (addExp2 3) 5 = ((addExp 2) 3) 5 = addExp 2 3 5
```

그런데 부분적용을 둘째, 셋째 인자에 대한 부분적용은 어떻게 할까?
이를 위해 **람다 추상화**를 이용하여 익명함수를 활용한다. 

### 람다 추상화와 함수 부분적용

부분적용 기법을 사용하여 새로운 함수를 정의하고자 할 수 있다. 

먼저 `add` 함수를 다시 살펴보자. 

```haskell
add x y = x + y
```

예를 들어 둘째 매개변수 `y`를 `3`로 고정하고 싶으면 다음과 같이 한다. 

In [19]:
addYIsThree = \x -> add x 3

Line 1: Redundant lambda
Found:
addYIsThree = \ x -> add x 3
Why not:
addYIsThree x = add x 3Line 1: Avoid lambda using `infix`
Found:
\ x -> add x 3
Why not:
(`add` 3)

#### 예제: `addExp` 함수의 부분적용

`addExp` 함수를 다시 살펴보자. 

```haskell
addExp x y z = (x + y)^z
```

이제 셋째 매개변수 `z`를 `2`로 고정하고 싶으면 다음과 같이 한다. 

In [20]:
addExpZIsTwo = \x y -> addExp x y 2

Line 1: Redundant lambda
Found:
addExpZIsTwo = \ x y -> addExp x y 2
Why not:
addExpZIsTwo x y = addExp x y 2

**주의:**
위 예제에서는 람다함수가 아닌 다른 방식으로 함수선언을 추천한다. 
익명함수의 유용성은 나중에 보다 재밌는 예제들을 살펴볼 때 알게 될 것이다.

### 함수의 좌결합성

이렇게 함수의 **부분적용**partial application이 허용되는 이유는
앞서 설명한 대로 유형의 우결합성에 근거한다. 
또한 함수의 부분 적용은 `add2`의 경우에서 보았듯이 **좌결합성**left associativity을 갖는다.

`f`가 함수 유형을 갖는다고 하자. 
그러면 다음이 성립한다. 

* `f :: T1 -> T2 -> T3` 인 경우

    ```haskell
    f t1 t2 = (f t1) t2
    ```

* `f :: T1 -> T2 -> T3 -> T4` 인 경우

    ```haskell
    f t1 t2 t3 = ((f t1) t2) t3
    ```

* `f :: T1 -> T2 -> T3 -> T4 -> T5` 인 경우

    ```haskell
    f t1 t2 t3 t4 = (((f t1) t2) t3) t4
    ```

* ...

### 부분함수

하스켈에서는 부분함수partial function도 허용된다. 
예를 들어, 리스트의 첫째 항목을 계산하는 `head`, 
첫째 항목을 제외한 나머지 항목들의 리스트를 계산하는 `tail` 등은 빈 리스트에 대애서는 정의되어 있지 않으며,
빈 리스트와 함께 적용할 경우 빈 리스트라고 경고문이 뜬다. 

In [21]:
head []

: 

In [22]:
tail []

: 

### 함수의 커리화/언커리화

`add` 함수를 두 개의 인자 대신 튜플로 하나를 사용하도록 수정해보자. 

In [23]:
addTuple :: (Integer, Integer) -> Integer
addTuple (n, m) = n + m

그러면 `add`와 `addTuple` 사실상 동일한 기능을 수행한다. 
즉, 임의의 정수 `n, m`에 대해 다음이 성립한다. 

```haskell
add n m = addTuple (n, m)
```

이와같이 튜플 유형의 인자를 여러 개 값들의 연속적인 적용으로 다루는 함수로 항상 바꿀 수 있으며,
이런 과정을 논리학자 Haskell Curry(하스켈 커리)의 이름을 따서 **커리화**,
역과정은 **언커리화**라고 부른다. 
예를 들어, `add`는 `addTuple`을 커리화한 함수이며, 
역으로 `addTuple`은 `add`를 역커리화한 함수이다. 

커리화/언커리화의 일반적인 형태는 다음과 같다. 
단, `fCu`는 커리화된 함수, `fUn`눈 언커리화된 함수를 가리킨다.

* `fUn :: (T1, T2) -> T3` 인 경우
    ```haskell
    fCu :: T1 -> T2 -> T3
    fCu t1 t2 = fUn (t1, t2)
    ```
    
* `fUn :: (T1, T2, T3) -> T4` 인 경우
    ```haskell
    fCu :: T1 -> T2 -> T3 -> T4
    fCu t1 t2 t3 = fUn (t1, t2, t3)
    ```
    
* `fUn :: (T1, T2, T3, T4) -> T5` 인 경우
    ```haskell
    fCu :: T1 -> T2 -> T3 -> T4 -> T5
    fCu t1 t2 t3 t4 = fUn (t1, t2, t3, t4)
    ```

* ...